In [102]:
import scala.util.Random
import java.time._
import org.apache.spark.sql.functions._

import scala.util.Random
import java.time._
import org.apache.spark.sql.functions._


In [103]:
var df = spark.read.format("csv").option("header", "true").load("weatherbaloon.csv")

df: org.apache.spark.sql.DataFrame = [timestamp: string, temperature: string ... 2 more fields]


In [104]:
df.printSchema()

root
 |-- timestamp: string (nullable = true)
 |-- temperature: string (nullable = true)
 |-- location: string (nullable = true)
 |-- observatory: string (nullable = true)



In [105]:
df = df.withColumn("temperature",col("temperature").cast("float"))

df: org.apache.spark.sql.DataFrame = [timestamp: string, temperature: float ... 2 more fields]


In [106]:
df.printSchema()

root
 |-- timestamp: string (nullable = true)
 |-- temperature: float (nullable = true)
 |-- location: string (nullable = true)
 |-- observatory: string (nullable = true)



In [107]:
def readYear(date: String): Int = {
    LocalDateTime.parse(date).getYear()
}

readYear: (date: String)Int


In [108]:
def readMonth(date: String): Int = {
    LocalDateTime.parse(date).getMonthValue()
}

readMonth: (date: String)Int


In [109]:
def readDay(date: String): Int = {
    LocalDateTime.parse(date).getDayOfMonth()
}

readDay: (date: String)Int


In [110]:
val readYearUDF = udf(readYear(_))
val readMonthUDF = udf(readMonth(_))
val readDayUDF = udf(readDay(_))

readYearUDF: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$4512/0x000000084185e840@3c2b2c1e,IntegerType,List(Some(class[value[0]: string])),Some(class[value[0]: int]),None,false,true)
readMonthUDF: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$4513/0x000000084185f840@468457cb,IntegerType,List(Some(class[value[0]: string])),Some(class[value[0]: int]),None,false,true)
readDayUDF: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$4514/0x000000084178a040@6787c19a,IntegerType,List(Some(class[value[0]: string])),Some(class[value[0]: int]),None,false,true)


In [111]:
df = df.withColumn("year",readYearUDF(col("timestamp")))

df: org.apache.spark.sql.DataFrame = [timestamp: string, temperature: float ... 3 more fields]


In [112]:
df = df.withColumn("month",readMonthUDF(col("timestamp")))

df: org.apache.spark.sql.DataFrame = [timestamp: string, temperature: float ... 4 more fields]


In [113]:
df = df.withColumn("day",readDayUDF(col("timestamp")))

df: org.apache.spark.sql.DataFrame = [timestamp: string, temperature: float ... 5 more fields]


In [114]:
df.show()

+--------------------+-----------+--------------------+-----------+----+-----+---+
|           timestamp|temperature|            location|observatory|year|month|day|
+--------------------+-----------+--------------------+-----------+----+-----+---+
|2021-11-07T20:33:...|  35.118458|-74.940826,63.670044|         AF|2021|   11|  7|
|2021-05-15T12:27:...|  14.567891|-10.101830,-44.23...|         CD|2021|    5| 15|
|2021-11-16T14:44:...|  -7.337278|-56.515732,-9.568994|         PR|2021|   11| 16|
|2021-06-09T21:23:...|  -7.251513|  3.590960,15.695739|        NaN|2021|    6|  9|
|2021-07-07T03:23:...|  40.985325|-43.057827,-74.82...|         MM|2021|    7|  7|
|2021-09-17T13:51:...|  -2.854151|   3.497397,5.322024|         BW|2021|    9| 17|
|2021-05-01T09:59:...|   3.016435|-27.532867,22.758667|         PR|2021|    5|  1|
|2021-12-24T20:36:...|   7.473855| 42.925018,16.073051|         GH|2021|   12| 24|
|2021-07-02T04:57:...|        0.0|  7.037897,89.790657|         KE|2021|    7|  2|
|202

In [117]:
df.filter($"temperature" !== "NaN").select(avg($"temperature")).show()

+-----------------+
| avg(temperature)|
+-----------------+
|5.587163308087518|
+-----------------+



In [116]:
df.filter($"month" === 2).filter($"temperature" !== "NaN").select(avg($"temperature")).show()

+----------------+
|avg(temperature)|
+----------------+
|            null|
+----------------+

